## import libararies

### NEED TO DO
 1. Hyperparameter Tuning
 2. Feature Importance - using 'rf_regressorfeature_importances_'
 3. Visualization

In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

##Load and prepare data

In [16]:
ssr_data = pd.read_csv("ssr_data.csv")

# Convert the dictionary into DataFrame
df = pd.DataFrame(ssr_data)

In [17]:
df = df.head(10000)

In [18]:
df

,Vdc,Tem,wr,Va,Vb,Vc,Ia,Ib,Ic,P(MW),Q(Mvar),Va_ssr,Vb_ssr,Vc_ssr,Ia_ssr,Ib_ssr,Ic_ssr
0,1150.000000,0.000000e+00,1.200000,0.000124,-0.000126,0.000002,-0.000260,-0.000259,0.000518,0.000000,0.000000,1075.041387,592.453390,482.587997,1098.891838,49.195441,1148.087278
1,1150.000000,0.000000e+00,1.200000,-0.004193,-0.004729,0.008922,-0.010945,-0.011664,0.022608,-84.423289,-62.786637,1075.577984,592.006626,483.690890,1093.460576,37.557874,1080.618548
2,1150.000001,-1.610000e-11,1.200054,-0.009945,-0.011039,0.020984,-0.021335,-0.023251,0.044586,-84.423289,-62.786637,1076.792216,594.959407,482.442549,1083.913864,151.144782,1119.426185
3,1150.000222,4.020000e-07,1.200080,-0.020186,-0.021877,0.042063,-0.034569,-0.038877,0.073447,-84.423289,-62.786637,1079.070323,598.587611,481.802231,1067.930438,142.552040,1114.761706
4,1149.998881,3.050000e-06,1.200107,-0.034121,-0.036223,0.070343,-0.046921,-0.054579,0.101500,-84.423289,-62.786637,1082.309390,602.121796,482.965761,1043.137930,238.107821,1068.062591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1150.315963,-5.779545e-01,1.191448,-0.108711,-0.769009,0.877720,0.182651,0.562984,-0.745635,73.444030,9.303706,1.591739,8.028980,9.603898,13.733010,6.003655,7.729897
9996,1150.316072,-5.779707e-01,1.191454,-0.090746,-0.779610,0.870356,0.168358,0.573030,-0.741388,73.446016,9.302430,1.578650,8.027069,9.590349,13.732486,6.002964,7.730064
9997,1150.316181,-5.779870e-01,1.191459,-0.072750,-0.789932,0.862682,0.154004,0.582872,-0.736876,73.448006,9.301158,1.574978,8.011693,9.571979,13.730636,6.002581,7.728620
9998,1150.316289,-5.780033e-01,1.191465,-0.054728,-0.799973,0.854701,0.139593,0.592507,-0.732101,73.449999,9.299890,1.576829,7.988061,9.550125,13.730272,6.001926,7.728919


In [19]:
df.dtypes

Vdc        float64
Tem        float64
wr         float64
Va         float64
Vb         float64
Vc         float64
Ia         float64
Ib         float64
Ic         float64
P(MW)      float64
Q(Mvar)    float64
Va_ssr     float64
Vb_ssr     float64
Vc_ssr     float64
Ia_ssr     float64
Ib_ssr     float64
Ic_ssr     float64
dtype: object

In [20]:
df.isnull().sum()

Vdc        0
Tem        0
wr         0
Va         0
Vb         0
Vc         0
Ia         0
Ib         0
Ic         0
P(MW)      0
Q(Mvar)    0
Va_ssr     0
Vb_ssr     0
Vc_ssr     0
Ia_ssr     0
Ib_ssr     0
Ic_ssr     0
dtype: int64

In [21]:
df = df.dropna()

In [22]:
df.isnull().sum()

Vdc        0
Tem        0
wr         0
Va         0
Vb         0
Vc         0
Ia         0
Ib         0
Ic         0
P(MW)      0
Q(Mvar)    0
Va_ssr     0
Vb_ssr     0
Vc_ssr     0
Ia_ssr     0
Ib_ssr     0
Ic_ssr     0
dtype: int64

In [23]:
# Remove three columns as index base
df.drop(df.columns[[12, 13, 14, 15, 16]], axis=1, inplace=True)

In [24]:
df.head()

,Vdc,Tem,wr,Va,Vb,Vc,Ia,Ib,Ic,P(MW),Q(Mvar),Va_ssr
0,1150.000000,0.000000e+00,1.200000,0.000124,-0.000126,0.000002,-0.000260,-0.000259,0.000518,0.000000,0.000000,1075.041387
1,1150.000000,0.000000e+00,1.200000,-0.004193,-0.004729,0.008922,-0.010945,-0.011664,0.022608,-84.423289,-62.786637,1075.577984
2,1150.000001,-1.610000e-11,1.200054,-0.009945,-0.011039,0.020984,-0.021335,-0.023251,0.044586,-84.423289,-62.786637,1076.792216
3,1150.000222,4.020000e-07,1.200080,-0.020186,-0.021877,0.042063,-0.034569,-0.038877,0.073447,-84.423289,-62.786637,1079.070323
4,1149.998881,3.050000e-06,1.200107,-0.034121,-0.036223,0.070343,-0.046921,-0.054579,0.101500,-84.423289,-62.786637,1082.309390


In [25]:

# Assuming you have a DataFrame 'df' with features and target variable
X = df.drop('Va_ssr', axis=1)
y = df['Va_ssr']


In [26]:
X.head()

,Vdc,Tem,wr,Va,Vb,Vc,Ia,Ib,Ic,P(MW),Q(Mvar)
0,1150.000000,0.000000e+00,1.200000,0.000124,-0.000126,0.000002,-0.000260,-0.000259,0.000518,0.000000,0.000000
1,1150.000000,0.000000e+00,1.200000,-0.004193,-0.004729,0.008922,-0.010945,-0.011664,0.022608,-84.423289,-62.786637
2,1150.000001,-1.610000e-11,1.200054,-0.009945,-0.011039,0.020984,-0.021335,-0.023251,0.044586,-84.423289,-62.786637
3,1150.000222,4.020000e-07,1.200080,-0.020186,-0.021877,0.042063,-0.034569,-0.038877,0.073447,-84.423289,-62.786637
4,1149.998881,3.050000e-06,1.200107,-0.034121,-0.036223,0.070343,-0.046921,-0.054579,0.101500,-84.423289,-62.786637


In [27]:
y.head()

0    1075.041387
1    1075.577984
2    1076.792216
3    1079.070323
4    1082.309390
Name: Va_ssr, dtype: float64

## Split Data into Training and Testing Sets

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Initialize the Regression model


In [29]:
# Define the neural network architecture
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Assuming a regression task with one output node
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


2023-11-01 13:30:23.551620: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Train the model

In [30]:

# Train the model with backpropagation
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
200/200 [==============================] - 2s 7ms/step - loss: 566613.6250 - val_loss: 234810.0156
Epoch 2/50
200/200 [==============================] - 1s 5ms/step - loss: 513947.7188 - val_loss: 195361.7344
Epoch 3/50
200/200 [==============================] - 1s 5ms/step - loss: 484508.1250 - val_loss: 183422.2812
Epoch 4/50
200/200 [==============================] - 1s 6ms/step - loss: 478647.9688 - val_loss: 181961.1562
Epoch 5/50
200/200 [==============================] - 1s 6ms/step - loss: 478528.5938 - val_loss: 181563.1875
Epoch 6/50
200/200 [==============================] - 2s 8ms/step - loss: 476294.2500 - val_loss: 180322.0781
Epoch 7/50
200/200 [==============================] - 1s 5ms/step - loss: 476078.4375 - val_loss: 181283.5000
Epoch 8/50
200/200 [==============================] - 1s 4ms/step - loss: 475445.8750 - val_loss: 180120.0469
Epoch 9/50
200/200 [==============================] - 1s 5ms/step - loss: 474228.1562 - val_loss: 178461.7656
Epoch 10/5

## Make Prediction

In [31]:

# # Make predictions on the test set
# y_pred = model.predict(X_test)


## Evaluate the Model

In [33]:
## Evaluate the model on the test data
results = model.evaluate(X_test, y_test)

# Print the test loss and accuracy (or other relevant metrics)
# print("Test Loss:", results[0])
# print("Test Accuracy:", results[1])
print(results)

63/63 [==============================] - 0s 3ms/step - loss: 316907.9688
316907.96875


In [ ]:
import math

rmse = math.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)


In [ ]:
# input_data = (1154.950194,0.383471064,1.067522947,-0.084752863,-0.385761442,0.470514305,1.743972979,-1.994108981,0.250136002,203.5984165,-8.976845521)
# # 0.5110339154585591

# ## converting input_data to numpy array
# input_data_as_numpy_array = np.asarray(input_data)

# ## reshape np array as we are predicting for one instance
# input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)


# y_pred = linear_reg.predict(input_data_reshaped)

In [ ]:
# print(y_pred)